In [1]:
import numpy as np
from typing import *

# Рассказываю, как работает вообще тема вся эта с вычислением уравнения
## Лайк за красоту решения!

Во-первых, обозначим аргументы - у нас есть:
1. интеграл определенный на области [a, b]
2. вырожденное ядро K(x, y) - здесь kernel_func
3. некоторая известная функция возмущения f(x) - здесь func или f
4. некоторая неизвестная функция u(x) - здесь никак не обозначена
5. некоторый параметр lambda - без понятия как в md символы особые вставлять

___
Все это добро предстает в виде интегральной функции, которое необходимо перевести в дискретный вид, в котором в свою очередь аргументы считаются по области Xi = a + i * h + h / 2, где h = |b - a| / N, где N - число разбиение отрезка [a, b]
___
Дальнейшие преобразования позволяют представить все выражение в матричной форме (I + K) * u = f,
откуда как раз можно сделать замену (I + K) = A и получить стандартное матричное уравнение Au = f

## Инициализируем значения на отрезке

Функция принимает N - число разбиений, a - левую границу и b - правую границу отрезка

На выходе получется кортеж с двумя параметрами:
1. значения X по числу разбиений
2. знаение шага разбиения h

In [2]:
def init_x_h(N: float, a: float, b: float) -> tuple[Sequence[float], float]:
    h = abs(b - a) / N
    x = [i * h  + h / 2 + a for i in range(N)]
    return x, h

## Инициализация аргумента (I + K)

Поскольку вырожденное ядро - функция, то на некотором определенном наборе X можно будет посчитать значения функции в конкретных точках

Затем можно будет добавить значение единичной матрицы и считать аргумент готовым к исполнению

Функция принимает в себя уже известные арргументы N, a, b, x и h, а также еще и функцию6 представляющую ядро - kernel_func

Возвращает матрицу значений **(I + K)**, которую мы будем называть **A** в дальнейшем

In [3]:
def init_a(
            N: float,
            a: float,
            b: float,
            kernel_func: Callable[[float, float], float],
            x: Sequence[float],
            h: float) -> np.matrix:
    kernel_func_h = lambda x, y: kernel_func(x, y) * h
    kernel = [ [kernel_func_h(i, j) for j in x] for i in x ]
    return np.identity(N) + np.matrix(kernel)


## Функция инициализации второго аргумента, в нашем случае вектора f функции возмущения

По аналогии с ядром выичисляется на наборе X

Приниамет следущие аргументы:
1. функцию вычисления f - func
2. вектор X

Возвращает вектор типа np.array вычисленных значений f

In [4]:
def init_b(func: Callable[[float], float], x: Sequence[float]) -> np.array:
    return np.array([func(i) for i in x])

## Функция создания вычисляемого выражения

Принимает в себя все определенные аргументы изначального уравнения:
1. число разбиений - N
2. левую границу участка - a
3. правую границу участка - b
4. функцию ядра - kernel_func
5. функцию внешнего возмущения - func

Возвращает функцию, не принимающую аргументов, вызов которой запускает процесс вычисления, основанный на np.linalg.solve

In [5]:
def init_equation(
            N: float,
            a: float,
            b: float,
            kernel_func: Callable[[float, float], float],
            func: Callable[[float], float]) -> Callable[[], np.array]:
    x, h = init_x_h(N, a, b)
    A = init_a(N, a, b, kernel_func, x, h)
    B = init_b(func, x)
    return A, B

In [6]:
a = 0
b = 1
N = 3000
kernel_func = lambda x, y: x * y
func = lambda x: x

In [7]:
equation = init_equation(N, a, b, kernel_func, func)
A_, B_ = equation

In [8]:
f"Абсолютная ошибка при решении np.linalg.solve: {np.linalg.norm(A_ @ np.linalg.solve(A_, B_) - B_)}"

'Абсолютная ошибка при решении np.linalg.solve: 3.8057460659108957e-14'

In [9]:
import inspect

import numpy as np
from IPython.display import Markdown as md

from src.equation import init_equation
from src.methods import simple_iteration

## 2)

## 3) Итерационный метод простых итераций для решения СЛАУ

In [10]:
md(f"""
### Реализованный метод простой итерации (см. [src/methods.py](src/methods.py))):

```python
{inspect.getsource(simple_iteration)}
```
""")


### Реализованный метод простой итерации (см. [src/methods.py](src/methods.py))):

```python
def simple_iteration(A: np.matrix, f: np.ndarray, epsilon: float = 0.0001) -> Generator[np.ndarray, None, None]:
    N = A.shape[0]
    u = np.random.random(N)
    u_old = np.random.random(N)

    B = np.eye(N) - A
    A = A / np.max(np.abs(A))
    f = f / np.max(np.abs(A))
    while np.linalg.norm(u - u_old) / np.linalg.norm(f) > epsilon:
        u_old = u
        u = np.dot(B, u) + f
        u = np.squeeze(np.asarray(u))
        yield u

```


In [11]:
equation = init_equation(3)
A_, B_ = equation

In [12]:
np.random.seed(1000 - 7)

iteration_history = list(simple_iteration(A_, B_))
f"Решено за {len(iteration_history)} шагов"

'Решено за 9 шагов'

In [13]:
import plotly.graph_objects as go


go.Figure(
    data=[
        go.Scatter(
            x=np.arange(len(iteration_history)) + 1,
            y=[
                np.linalg.norm(np.linalg.solve(A_, B_) - iteration_solve)
                for iteration_solve in iteration_history
            ],
            mode="lines+markers",
        ),
    ],
    layout=go.Layout(
        title=go.layout.Title(text="Отклонение решения итерационного метода от \'точного\' решения"),
        xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text="Количество итераций")),
        yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text="Норма разности решений")),
    ),
).show()

## 4) Сравнить численный и аналитический результаты

In [14]:
iteration_solution = iteration_history[-1]
anal_solution = np.linalg.solve(A_, B_)

In [15]:
delta = np.linalg.norm(iteration_solution - anal_solution) / np.linalg.norm(iteration_solution)
delta

1.2026861267156023e-05